Read cytobands(hg38) from a file, for each dataset, count the overlaps in each band than save to db.  
This is the later List version.
Current version.

In [32]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import copy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Script version

In [ ]:
db = MongoClient()['Rebased']['skin']
samples = []
for sam in db.find():
    samples.append(sam)

Bands info is manually parsed

In [33]:
# bands is a list of 24 lists, represents 24 chromosomes. Each sub-list stores the cytobands of that chromosome as another list
bands = []
for i in range(24):
    bands.append([])
    

with open('/Users/bogao/DataFiles/Data/genome/cytoBand_hg38.txt','r') as fin:
    for line in fin:
        tokens = line.strip().split('\t')
        if ('_' in tokens[0]) or ('M' in tokens[0]):
            continue
        elif 'X' in tokens[0]:
            chro = 22
        elif 'Y' in tokens[0]:
            chro = 23
        else:
            chro = int(tokens[0][3:])-1
        # index of counters: total_dup=4, total_del=5, dup_length=6, del_length=7, dup_count=8, del_count=9,   
        info = [int(tokens[1]),int(tokens[2]), *tokens[3:], *[0]*6]
        bands[chro].append(info)

        
    

Key part that aggregates values into bands.  
The the saved values = seg_value * overlapping_size.  
Later, ave_value = total_value/total_overlapping_size

In [ ]:
c = 1
for sam in samples:
    for seg in sam['segments']:
        if seg['chro'] == 'X':
            chro = 22
        elif seg['chro'] == 'Y':
            chro = 23
        else:
            chro = int(seg['chro'])-1
            
        # find all matching bands
        for b in bands[chro]:
            if (b[0] < seg['end']) and (b[1] >= seg['start']):
                size = min(b[1],seg['end']) - max(b[0],seg['start'])
                
                # dup
                if seg['value'] > 0:
                    # update total value
                    b[4] += size*seg['value']
                    # update total_length
                    b[6] += size
                    # update count
                    b[8] +=1
                # del
                else:
                    # update total value
                    b[5] += size*seg['value']
                    # update total_length
                    b[7] += size
                    # update count
                    b[9] +=1
    print(c, end='\r')
    c +=1
    
#     if c>1:
#         break

Flat the nested lists to a 1-d list

In [ ]:
bands_list = []
idx = 1
for ch in bands:
    if idx == 23:
        chro = 'chrX'
    elif idx == 24:
        chro = 'chrY'
    else:
        chro = ''.join(['chr', str(idx)])
    for b in ch:
        bands_list.append( [chro] + b)
    idx +=1

Use a Dataframe for easy transformation to dict, then save to db

In [ ]:
df_bands = pd.DataFrame(bands_list,columns=['chr','start','end','name','note','total_dup','total_del','dup_length','del_length','dup_count','del_count'])

skin_bands = df_bands.to_dict('records')
db = MongoClient()['Rebased']['bands_skin_original']
db.drop()
db.insert_many(skin_bands)

## The function version

In [49]:
def cytobandOverlap(cytobands, dbin, dbout, vtype='segments'):
    bands = copy.deepcopy(cytobands)
    c = 1
    for sam in dbin.find():
        if sam[vtype] != None:
            for seg in sam[vtype]:
                if seg['probes'] >4:
                    
                    if seg['value'] >2:
                        val = 2
                    else:
                        val = seg['value']
                    
                    if seg['chro'] == 'X':
                        chro = 22
                    elif seg['chro'] == 'Y':
                        chro = 23
                    else:
                        chro = int(seg['chro'])-1

                    # find all matching bands
                    for b in bands[chro]:
                        if (b[0] < seg['end']) and (b[1] >= seg['start']):
                            size = min(b[1],seg['end']) - max(b[0],seg['start'])

                            # dup
                            if seg['value'] > 0:
                                # update total value
                                b[4] += (2**(val+1)-2)*size
                                # update total_length
                                b[6] += size
                                # update count
                                b[8] +=1
                            # del
                            else:
                                # update total value
                                b[5] += (2**(val+1)-2)*size
                                # update total_length
                                b[7] += size
                                # update count
                                b[9] +=1
            print(c, end='\r')
            c +=1
      
    bands_list = []
    idx = 1
    for ch in bands:
        if idx == 23:
            chro = 'chrX'
        elif idx == 24:
            chro = 'chrY'
        else:
            chro = ''.join(['chr', str(idx)])
        for b in ch:
            bands_list.append( [chro] + b)
        idx +=1
    bands_df = pd.DataFrame(bands_list,columns=['chr','start','end','name','note','total_dup','total_del','dup_length','del_length','dup_count','del_count'])    
    bands_records = bands_df.to_dict('records')    
    dbout.drop()
    dbout.insert_many(bands_records)
    return bands

Testings

In [53]:
# ret = cytobandOverlap(bands, MongoClient()['Rebased']['skin'], MongoClient()['Rebased']['bands_skin_original'], 'segments')
# ret = cytobandOverlap(bands, MongoClient()['Rebased']['skin'], MongoClient()['Rebased']['bands_skin_normalized'], 'normalized')
# ret = cytobandOverlap(bands, MongoClient()['Rebased']['breast'], MongoClient()['Rebased']['bands_breast_original'], 'segments')
ret = cytobandOverlap(bands, MongoClient()['Rebased']['breast'], MongoClient()['Rebased']['bands_breast_normalized'], 'normalized')
ret = cytobandOverlap(bands, MongoClient()['Rebased']['ovary'], MongoClient()['Rebased']['bands_ovary_original'], 'segments')
ret = cytobandOverlap(bands, MongoClient()['Rebased']['ovary'], MongoClient()['Rebased']['bands_ovary_normalized'], 'normalized')